# Importation des Images et cration du jeu de données

In [1]:
pip install -r C:/Users/paulz/Desktop/projet_master/requierments.txt

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\paulz\Desktop\projet_master\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [2]:
# Importation des packages
import os
from PIL import Image, ImageEnhance
import cv2
import imutils
import numpy as np
from torchvision.transforms import v2

## Enhance Data: Brightness Modifier

In [3]:
# Modification de l'image: Luminosité
def apply_brightness_modif(img_file, folder_path):

    img = Image.open('flags/%s.png'%img_file).convert('RGBA')
    img.thumbnail((512,512))

    enhancer = ImageEnhance.Brightness(img)
    factor = [0.05, 0.2,0.5,1.5,2,3,4,6,8,10]
    for f in factor:
        # Si le facteur i est en dessous de 1 l'image est sous exposé, et inversemment
        bright_img = enhancer.enhance(f)
        bright_img.save('{0}/{1}_bright_{2}.png'.format(folder_path,img_file,f))

## Enhance Data: Noise Modifier

In [4]:
def add_gaussian_noise(img, mean = 0, std = 25):

    noise = np.random.normal(mean, std, img.shape).astype(np.uint8)
    noisy_img = cv2.add(img, noise)

    return noisy_img

def apply_noise_modif(img_file, folder_path):

    img = cv2.imread('flags/%s.png'%img_file)
    img = imutils.resize(img, width = 512)
    
    for i in range(1,11):
        std = round(10**(i/5))
        noisy_img = add_gaussian_noise(img,0,std)
    
        cv2.imwrite('{0}/{1}_noise_{2}.png'.format(folder_path,img_file,std), noisy_img)

## Enhance Data: Wave Modifier

In [5]:
def apply_wave_modif(img_file, folder_path):

    # Image test (Afrique du sud)
    img = Image.open('flags/%s.png'%img_file).convert('RGBA')
    img.thumbnail((512,512))

    for a in range(1,11):
        # modification du param exponentiel pour avoir des résultat plus "extrème"
        alpha = a**3

        # Applique la déformation suivant le niveau alpha
        elastic_transformer = v2.ElasticTransform(alpha=alpha)
        wavey_img = elastic_transformer(img)
        wavey_img.save('{0}/{1}_wave_{2}.png'.format(folder_path,img_file,alpha))

## Create Dataset

In [6]:
FLAG_LIST_DIR = 'flag_list.txt'
DATASET_DIR = 'C:/Users/paulz/Desktop/projet_master/input_data'

with open(FLAG_LIST_DIR, 'r') as f:
    flags = [line[:-1] for line in f]

selected_flags = np.random.choice(flags,10)

# Crée ou détruit le dossier d'inputs
if not os.path.exists(DATASET_DIR):
    os.makedirs(DATASET_DIR)       

for flag in selected_flags:

    flag_dir = '{0}/{1}'.format(DATASET_DIR,flag)

    if os.path.exists(flag_dir) == False:
        os.makedirs(flag_dir)

    apply_brightness_modif(flag,flag_dir)
    apply_noise_modif(flag,flag_dir)
    apply_wave_modif(flag,flag_dir)

print('Flags: ', selected_flags)

Flags:  ['ag' 'ph' 'qa' 'gh' 'fr' 'pt' 'sb' 'ru' 'ge' 'ne']


## KNN Classification

In [7]:
#package importation
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [8]:
def load_images(input_dir):
    images = []
    labels = []

    for folder in os.listdir(input_dir):

        label_folder = os.path.join(input_dir, folder)
        if os.path.isdir(label_folder):
            for filename in os.listdir(label_folder):
                img_path = os.path.join(label_folder, filename)
                img = cv2.imread(img_path)
                if img is not None:
                    images.append(img)
                    labels.append(folder)
    return images, labels

images, labels = load_images(DATASET_DIR)
print(len(images), labels)

300 ['ag', 'ag', 'ag', 'ag', 'ag', 'ag', 'ag', 'ag', 'ag', 'ag', 'ag', 'ag', 'ag', 'ag', 'ag', 'ag', 'ag', 'ag', 'ag', 'ag', 'ag', 'ag', 'ag', 'ag', 'ag', 'ag', 'ag', 'ag', 'ag', 'ag', 'fr', 'fr', 'fr', 'fr', 'fr', 'fr', 'fr', 'fr', 'fr', 'fr', 'fr', 'fr', 'fr', 'fr', 'fr', 'fr', 'fr', 'fr', 'fr', 'fr', 'fr', 'fr', 'fr', 'fr', 'fr', 'fr', 'fr', 'fr', 'fr', 'fr', 'ge', 'ge', 'ge', 'ge', 'ge', 'ge', 'ge', 'ge', 'ge', 'ge', 'ge', 'ge', 'ge', 'ge', 'ge', 'ge', 'ge', 'ge', 'ge', 'ge', 'ge', 'ge', 'ge', 'ge', 'ge', 'ge', 'ge', 'ge', 'ge', 'ge', 'gh', 'gh', 'gh', 'gh', 'gh', 'gh', 'gh', 'gh', 'gh', 'gh', 'gh', 'gh', 'gh', 'gh', 'gh', 'gh', 'gh', 'gh', 'gh', 'gh', 'gh', 'gh', 'gh', 'gh', 'gh', 'gh', 'gh', 'gh', 'gh', 'gh', 'ne', 'ne', 'ne', 'ne', 'ne', 'ne', 'ne', 'ne', 'ne', 'ne', 'ne', 'ne', 'ne', 'ne', 'ne', 'ne', 'ne', 'ne', 'ne', 'ne', 'ne', 'ne', 'ne', 'ne', 'ne', 'ne', 'ne', 'ne', 'ne', 'ne', 'ph', 'ph', 'ph', 'ph', 'ph', 'ph', 'ph', 'ph', 'ph', 'ph', 'ph', 'ph', 'ph', 'ph', 'ph', 'ph',

In [9]:
def preprocess_images(images, size=(64, 64)):
    preprocessed_images = []
    for img in images:
        # Resize image
        resized_img = cv2.resize(img, size)
        # Flatten the image
        flattened_img = resized_img.flatten()
        preprocessed_images.append(flattened_img)
    return np.array(preprocessed_images)

In [10]:
X = preprocess_images(images, (126,126))
y = np.array(labels)

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2,
                                                    random_state=1)

for i in range(1,10):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train, y_train)

    y_pred = knn.predict(X_test)
    accuracy = accuracy_score(y_test,y_pred)
    precision = precision_score(y_test,y_pred, average='weighted')
    recall = recall_score(y_test,y_pred, average='weighted')
    f1 = f1_score(y_test,y_pred, average='weighted')

    print(f'N-Neighbors: {i} | Accuracy: {accuracy*100:.2f}% | Precision: {precision*100:.2f}% | Recall: {recall*100:.2f}% | F1: {f1*100:.2f}%')

N-Neighbors: 1 | Accuracy: 93.33% | Precision: 94.03% | Recall: 93.33% | F1: 93.41%
N-Neighbors: 2 | Accuracy: 91.67% | Precision: 93.24% | Recall: 91.67% | F1: 91.94%
N-Neighbors: 3 | Accuracy: 91.67% | Precision: 93.24% | Recall: 91.67% | F1: 91.94%
N-Neighbors: 4 | Accuracy: 88.33% | Precision: 92.62% | Recall: 88.33% | F1: 89.50%
N-Neighbors: 5 | Accuracy: 88.33% | Precision: 92.62% | Recall: 88.33% | F1: 89.50%
N-Neighbors: 6 | Accuracy: 88.33% | Precision: 93.54% | Recall: 88.33% | F1: 89.87%
N-Neighbors: 7 | Accuracy: 88.33% | Precision: 92.14% | Recall: 88.33% | F1: 89.36%
N-Neighbors: 8 | Accuracy: 90.00% | Precision: 91.61% | Recall: 90.00% | F1: 90.24%
N-Neighbors: 9 | Accuracy: 88.33% | Precision: 90.25% | Recall: 88.33% | F1: 88.59%


# SVM

In [11]:
from sklearn import svm
from sklearn.preprocessing import LabelEncoder

In [12]:

images, labels = load_images(DATASET_DIR)
images = preprocess_images(images, (126,126))
labels = np.array(labels)

label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

X_train, X_test, y_train, y_test = train_test_split(images,
                                                    encoded_labels,
                                                    test_size=0.2,
                                                    random_state=1)

kernels = ('linear', 'poly', 'rbf', 'sigmoid')

for kernel in kernels:
    classifier = svm.SVC(kernel=kernel)
    classifier.fit(X_train, y_train)

    y_pred = classifier.predict(X_test)
    accuracy = accuracy_score(y_test,y_pred)
    precision = precision_score(y_test,y_pred, average='weighted')
    recall = recall_score(y_test,y_pred, average='weighted')
    f1 = f1_score(y_test,y_pred, average='weighted')

    print(f'kernel: {kernel} | Accuracy: {accuracy*100:.2f}% | Precision: {precision*100:.2f}% | Recall: {recall*100:.2f}% | F1: {f1*100:.2f}%')

kernel: linear | Accuracy: 93.33% | Precision: 94.94% | Recall: 93.33% | F1: 93.63%
kernel: poly | Accuracy: 90.00% | Precision: 94.62% | Recall: 90.00% | F1: 91.04%
kernel: rbf | Accuracy: 90.00% | Precision: 92.61% | Recall: 90.00% | F1: 90.41%
kernel: sigmoid | Accuracy: 35.00% | Precision: 55.75% | Recall: 35.00% | F1: 37.87%


c:\Users\paulz\Desktop\projet_master\venv\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# CNN

In [13]:
# Packages import
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import matplotlib.pyplot as plt

In [14]:
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    DATASET_DIR,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    DATASET_DIR,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 240 images belonging to 10 classes.
Found 60 images belonging to 10 classes.


In [15]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(train_generator.num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

c:\Users\paulz\Desktop\projet_master\venv\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
